In [1]:
# python version
import sys
print("Python version")
print (sys.version)

Python version
3.9.13 | packaged by conda-forge | (main, May 27 2022, 16:56:21) 
[GCC 10.3.0]


# Introduction plot

In [2]:
# The introduction are obtained using the matUtils in Usher.
INTRO_CSV_FILE = 'data/nexstrain_na_introductions.tsv'
# The number of COVID cases for a location is obatined from Our World in Data and Outbreak.
CASES_CSV_FILE = 'data/USA_US-CA_outbreakinfo_epidemiology_data_2022-07-28.tsv'
LOC = 'California'
ISO_CODE = 'USA'

In [3]:
import pandas as pd
import altair as alt
import numpy as np
import pycountry
from tqdm.notebook import tqdm
tqdm.pandas()

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Cases

In [4]:
df_cases = pd.read_csv(CASES_CSV_FILE, sep='\t')
df_cases.columns

Index(['_id', 'admin_level', 'confirmed', 'confirmed_doublingRate',
       'confirmed_numIncrease', 'confirmed_numIncrease_per_100k',
       'confirmed_per_100k', 'confirmed_rolling', 'confirmed_rolling_per_100k',
       'country_name', 'date', 'dead', 'dead_doublingRate', 'dead_numIncrease',
       'dead_numIncrease_per_100k', 'dead_per_100k', 'dead_rolling',
       'dead_rolling_per_100k', 'location_id', 'mostRecent', 'name', 'source'],
      dtype='object')

In [5]:
# Pre-process cases
# df_cases = df_cases.loc[df_cases['iso_code'] == ISO_CODE]
df_cases['date'] = pd.to_datetime(df_cases['date'])

# Smooth the data.
df_cases['confirmed_numIncrease'].fillna(method="bfill").fillna(0)
# rolling average of new cases.
df_cases['new_cases_rolling_avg'] = df_cases['confirmed_numIncrease'].rolling(7).mean()
df_cases['new_cases_rolling_avg'] = df_cases['new_cases_rolling_avg'].fillna(method="bfill").fillna(0)

In [6]:
# Plot a line plot depicting the number of cases per day for the given iso code.
cases_line_plot = alt.Chart(df_cases).mark_line().encode(
    x = alt.X('date:T', title="Date"),
    y = alt.Y('new_cases_rolling_avg:Q', axis=alt.Axis(title='Number of cases (Rolling average)', titleColor='#ff7f0e')),
    tooltip = ['date', 'new_cases_rolling_avg'],
    # color=alt.condition(interval, alt.value('#d62728'), alt.value('#1f77b4'))
    color = alt.value('#ff7f0e'),
)
cases_line_plot

alt.Chart(...)

## Introduction

In [7]:
df = pd.read_csv(INTRO_CSV_FILE, sep='\t')
df

,sample,introduction_node,introduction_rank,growth_score,earliest_date,latest_date,cluster_size,cluster_span,intro_confidence,parent_confidence,distance,origin_gap,region,origins,origins_confidence,mutation_path,meta_uncertainty
0,USA/WV-CDC-LC0773543/2022,West Virginia_node_2599,1,1.000000,2022-Jul-23,2022-Jul-23,1,1,1.0,0.333333,1,1,West Virginia,Tennessee,0.333333,C14925T<,0.161432
1,USA/WV-CDC-LC0777739/2022,West Virginia_node_2187,2,1.000000,2022-Jul-26,2022-Jul-26,1,5,1.0,0.444444,5,2,West Virginia,Vermont,0.555556,"C18582T,A19440G<",0.122066
2,USA/WV-CDC-LC0771158/2022,West Virginia_node_2568,3,0.500000,2022-Jul-21,2022-Jul-21,1,3,1.0,0.000000,3,1,West Virginia,Virginia,1.000000,C23707T<,0.000739
3,USA/WV-CDC-LC0726455/2022,West Virginia_node_2586,4,0.250000,2022-Jul-05,2022-Jul-05,1,2,1.0,0.125000,2,1,West Virginia,Indiana,0.250000,T492C<,0.055676
4,USA/WV-CDC-LC0738627/2022,West Virginia_node_2134,5,0.250000,2022-Jul-08,2022-Jul-08,1,4,1.0,0.076923,4,1,West Virginia,North Dakota,0.375000,A23440G<,0.019668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3246,MWI/MLW001170/2022,Malawi_node_2204,1,0.078567,2022-Mar-29,2022-Mar-29,2,7,1.0,0.000000,4,1,Malawi,Kenya,1.000000,T3802C<,0.001354
3247,MWI/MLW001207/2022,Malawi_node_2204,1,0.078567,2022-Mar-29,2022-Mar-29,2,7,1.0,0.000000,3,1,Malawi,Kenya,1.000000,T3802C<,0.001363
3248,MWI/MLW001213/2022,Malawi_node_1596,2,0.078567,2022-Mar-29,2022-Mar-29,2,4,1.0,0.000000,3,1,Malawi,Georgia,1.000000,G22813T<,0.000993
3249,MWI/MLW001208/2022,Malawi_node_1596,2,0.078567,2022-Mar-29,2022-Mar-29,2,4,1.0,0.000000,1,1,Malawi,Georgia,1.000000,G22813T<,0.000822


In [8]:
df['region'].unique()

array(['West Virginia', 'Colombia', 'Xieng Khouang', 'Schaffhausen',
       'Sonora', 'Sierra Leone', 'Bangkok', 'Friuli Venezia Giulia',
       'Mexico', 'Czech Republic', 'Utah', 'Hong Kong', 'USA', 'Zimbabwe',
       'Hovedstaden', 'China', 'Iraq', 'Alaska', 'Indonesia',
       'New Zealand', 'Lebanon', 'Vestland', 'Northern Mariana Islands',
       'Seoul', 'Suez', 'Italy', 'Guangdong', 'Germany', 'Vojvodina',
       'Switzerland', 'Arkansas', 'Rhode Island', 'Cameroon',
       'New Hampshire', 'Baja California', 'New Mexico', 'Israel',
       'Zürich', 'Ontario', 'Liechtenstein', 'Bern', 'Schleswig-Holstein',
       'Aargau', 'Iowa', 'North Rhine Westphalia', 'Turkey', 'Zhejiang',
       "Provence-Alpes-Côte d'Azur", 'Vaud', 'Geneva', 'Uganda',
       'Saskatchewan', 'Taiwan', 'Egypt', 'Jamaica', 'Riyadh',
       'Dominican Republic', 'Valais', 'Peru', 'Ticino', 'Florida',
       'Illinois', 'Vientiane', 'American Samoa', 'Massachusetts',
       'Puerto Rico', 'Delaware', 'Queensl

In [9]:
# Remove invalid dates
df = df[df['earliest_date'] != 'no-valid-date']
df = df[df['earliest_date'] != '?']

In [10]:
# Calculate the mean of earliest_date & latest_date.
def mean_date(earliest_date, latest_date):
    return (np.array([earliest_date, latest_date], dtype='datetime64[s]').view('i8').mean().astype('datetime64[s]'))

                      
df['earliest_date'] = pd.to_datetime(df['earliest_date'])
df['latest_date'] = pd.to_datetime(df['latest_date'])
df['mean_date'] = df.progress_apply(lambda x: mean_date(x['earliest_date'], x['latest_date']), axis = 1)
# df['mean_date'] = df['latest_date']

  0%|          | 0/3251 [00:00<?, ?it/s]

In [11]:
df_filterd = df[df['region'].str.contains(LOC)]
# Deal with missing datapoints in the source dataframe
source = df_filterd.groupby(['mean_date', 'origins']).size()
source = source.to_frame(name="count").reset_index()
source_df = pd.DataFrame({'mean_date': pd.date_range(sorted(list(source['mean_date']))[0], sorted(list(source['mean_date']))[-1])})
source_df = source_df.merge(source, on='mean_date', how='left')
source_df['count'] = source_df['count'].fillna(0)

In [12]:
conf_inter_df = df_filterd
conf_inter_df['earliest_date'] = pd.to_datetime(conf_inter_df['earliest_date'])
conf_inter_df['latest_date'] = pd.to_datetime(conf_inter_df['latest_date'])
conf_inter_df['interval'] = (conf_inter_df['latest_date'] - conf_inter_df['earliest_date']).dt.days

/tmp/ipykernel_246999/1985084789.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conf_inter_df['earliest_date'] = pd.to_datetime(conf_inter_df['earliest_date'])
/tmp/ipykernel_246999/1985084789.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conf_inter_df['latest_date'] = pd.to_datetime(conf_inter_df['latest_date'])
/tmp/ipykernel_246999/1985084789.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [13]:
legend_selection = alt.selection_multi(fields=['origins'], bind='legend')

chart = alt.Chart(conf_inter_df).mark_circle().encode(
    x=alt.X("mean_date", title="Date"),
    y=alt.Y("interval", title="Confidence interval(days)"),
    size='count(interval):Q',
    opacity=alt.condition(legend_selection, alt.value(1), alt.value(0.2)),
    color='origins',
    tooltip=['mean_date', 'interval', 'origins']
).add_selection(
    legend_selection
).interactive()
chart

alt.Chart(...)

In [14]:
# Plot number of introductions over time for a given location
brush = alt.selection_interval(
            encodings=["x"], mark=alt.BrushConfig(fill="green")
        )

bars = (
    alt.Chart(source_df)
    .mark_bar()
    .encode(
        x=alt.X("sum(count)", title="Number of Introductions"),
        y=alt.Y("origins", title="Origins", sort='-x'),
        tooltip=["sum(count)"],
    )
    .transform_filter(brush)
)

intro_base = alt.Chart(source_df).mark_line().encode(
    x = alt.X('mean_date', title="Date"),
    y = alt.Y('sum(count):Q',axis=alt.Axis(title='Number of Introducitons')),
    tooltip = ['mean_date', 'sum(count):Q']
)

intro_cases = alt.layer(intro_base, cases_line_plot).resolve_scale(
            y='independent'
        ).properties(width=800, height=300)

intro_upper = intro_cases.encode(
    alt.X('mean_date:T', scale=alt.Scale(domain=brush))
).properties(
    title = 'Selected date range'
)

intro_lower = intro_cases.properties(
    height=60,
    title = 'Select date range'
).add_selection(brush)

(intro_upper & intro_lower) | bars

alt.HConcatChart(...)